## Data loading

In [104]:
import csv
file = open('DataAnalyst.csv', encoding="utf8")
file_reader = csv.DictReader(file)
data = []
for row in file_reader:
    data.append(row)    
file.close()

In [105]:
def print_no_job_description(data):
    for i in range(0, 10):
        row = data[i]
        for key in row:
            if key != 'Job Description':
                print(str(key) + ':' + str(row[key]))
        print('\n')    

## Data cleaning

In [106]:
def remove_duplicates(data):
    unique = []
    for row in data:
        if row not in unique:
            unique.append(row)
    return unique        

In [107]:
print(len(data))
data = remove_duplicates(data)
print(len(data))

2253
2253


## Check column headers

In [108]:
def get_all_headers(data):
    list_of_keys = set()
    for dictionary in data:
        list_of_keys.update(dictionary.keys())
    return list_of_keys

In [109]:
get_all_headers(data)

{' ',
 'Company Name',
 'Competitors',
 'Easy Apply',
 'Founded',
 'Headquarters',
 'Industry',
 'Job Description',
 'Job Title',
 'Location',
 'Rating',
 'Revenue',
 'Salary Estimate',
 'Sector',
 'Size',
 'Type of ownership'}

## Fix empty headers

In [110]:
def rename_empty_header(data, old_header, replacement_header):
    for row in data:
        row[replacement_header] = row[old_header]       
        del row[old_header]       

In [111]:
rename_empty_header(data, ' ', 'Index')

In [112]:
get_all_headers(data)

{'Company Name',
 'Competitors',
 'Easy Apply',
 'Founded',
 'Headquarters',
 'Index',
 'Industry',
 'Job Description',
 'Job Title',
 'Location',
 'Rating',
 'Revenue',
 'Salary Estimate',
 'Sector',
 'Size',
 'Type of ownership'}

## Fix invalid values

In [113]:
# invalid values means None, empty string or -1
def count_invalid_values_for_column(data, col):
    count_invalid_data = 0 
    for row in data:
        if col not in row:
            continue
        value = row[col]
        if value == None or value == '' or value == '-1':
            count_invalid_data += 1
    return count_invalid_data       

In [114]:
def count_invalid_data(data):
    all_headers = get_all_headers(data)
    invalid_values_per_column = {}
    for header in all_headers:
        invalid_values_per_column[header] = count_invalid_values_for_column(data, header)
    return invalid_values_per_column

In [115]:
count_invalid_data(data)

{'Easy Apply': 2173,
 'Industry': 353,
 'Revenue': 163,
 'Company Name': 1,
 'Location': 0,
 'Size': 163,
 'Sector': 353,
 'Competitors': 1732,
 'Index': 0,
 'Type of ownership': 163,
 'Job Title': 0,
 'Founded': 660,
 'Salary Estimate': 1,
 'Rating': 272,
 'Job Description': 0,
 'Headquarters': 172}

## Ensure consistent naming for invalid values

In [116]:
def find_unique_values_for_column(data, col):
    list_of_values = set()
    for row in data:
        if col not in row:
            continue
        list_of_values.add(row[col])
    return list_of_values

In [117]:
find_unique_values_for_column(data, 'Easy Apply')

{'-1', 'True'}

In [118]:
def replace_specific_value_with_specified(data, col, old_value, new_value):
    cleaned_data = data.copy()
    for dictionary in cleaned_data:
        if col not in row:
            continue
        if dictionary[col] == old_value:
            dictionary[col] = new_value           
    return cleaned_data        

In [119]:
data = replace_specific_value_with_specified(data, 'Easy Apply', '-1', 'NA')

In [120]:
data = replace_specific_value_with_specified(data, 'Competitors', '-1', 'NA')

In [121]:
data = replace_specific_value_with_specified(data, 'Rating', '-1', 'NA')

In [122]:
data = replace_specific_value_with_specified(data, 'Headquarters', '-1', 'NA')

In [123]:
data = replace_specific_value_with_specified(data, 'Industry', '-1', 'NA')

In [124]:
data = replace_specific_value_with_specified(data, 'Type of ownership', '-1', 'NA')

In [125]:
data = replace_specific_value_with_specified(data, 'Type of ownership', 'Unknown', 'NA')

In [126]:
data = replace_specific_value_with_specified(data, 'Company Name', '', 'NA')

In [127]:
data = replace_specific_value_with_specified(data, 'Salary Estimate', '-1', 'NA')

In [128]:
data = replace_specific_value_with_specified(data, 'Founded', '-1', 'NA')

In [129]:
data = replace_specific_value_with_specified(data, 'Sector', '-1', 'NA')

In [130]:
data = replace_specific_value_with_specified(data, 'Revenue', '-1', 'NA')

In [131]:
data = replace_specific_value_with_specified(data, 'Revenue', 'Unknown / Non-Applicable', 'NA')

In [132]:
data = replace_specific_value_with_specified(data, 'Size', '-1', 'NA')

In [133]:
data = replace_specific_value_with_specified(data, 'Size', 'Unknown', 'NA')

In [134]:
def count_NA_values_for_specified_column(data, col):
    count_invalid_data = 0 
    for row in data:
        if col not in row:
            continue
        if row[col] == 'NA':
            count_invalid_data += 1
    return count_invalid_data       

In [135]:
def count_NA_values(data):
    all_headers = get_all_headers(data)
    NA_values_per_column = {}
    for header in all_headers:
        NA_values_per_column[header] = count_NA_values_for_specified_column(data, header)
    return NA_values_per_column

In [136]:
count_NA_values(data)

{'Easy Apply': 2173,
 'Industry': 353,
 'Revenue': 778,
 'Company Name': 1,
 'Location': 0,
 'Size': 205,
 'Sector': 353,
 'Competitors': 1732,
 'Index': 0,
 'Type of ownership': 179,
 'Job Title': 0,
 'Founded': 660,
 'Salary Estimate': 1,
 'Rating': 272,
 'Job Description': 0,
 'Headquarters': 172}

## Clean Company Name column

In [137]:
comp = find_unique_values_for_column(data, 'Company Name')

In [138]:
# remove rating from clumn 'Company Name'
def clean_column_company_name(data):
    cleaned_data = data.copy()
    key = 'Company Name'
    for row in cleaned_data:
        if key not in row:
            continue
        parts = str(row[key]).split('\n')
        row[key] = parts[0].strip()    
    return cleaned_data   

In [139]:
data = clean_column_company_name(data)

In [140]:
comp = find_unique_values_for_column(data, 'Company Name')

## Clean Size column

In [141]:
find_unique_values_for_column(data, 'Size')

{'1 to 50 employees',
 '10000+ employees',
 '1001 to 5000 employees',
 '201 to 500 employees',
 '5001 to 10000 employees',
 '501 to 1000 employees',
 '51 to 200 employees',
 'NA'}

In [142]:
import re
def extract_max_no_of_employees(data):
    cleaned_data = data.copy()
    key = 'Size'
    new_key = 'Max size'
    for row in cleaned_data:
        if key not in row:
            continue
        if '10000+' in row[key]:
            row[new_key] = 10000
        else: 
            value = re.search(r'(\d+)(\sto\s)*(\d*)(.*)', row[key], re.IGNORECASE)
            if value:    
                row[new_key] = int(value.group(3))    
            else:
                row[new_key] = 'NA'
    return cleaned_data

In [143]:
data = extract_max_no_of_employees(data)

In [144]:
find_unique_values_for_column(data, 'Max Size')

set()

## Extract min and max salary and calculate average salary

In [146]:
import re
def extract_min_and_max_salary_as_new_columns(data):
    key = 'Salary Estimate'
    for row in data:
        if key not in row:
            continue
        value = re.search(r"\$(\d+)(.*)\$(\d+)(.*)", row[key], re.IGNORECASE)
        if value:
            row['Min salary'] = int(value.group(1))
            row['Max salary'] = int(value.group(3))
    return data  

In [147]:
data = extract_min_and_max_salary_as_new_columns(data)

In [148]:
def calculate_average_salary_as_new_column(data):
    cleaned_data = data.copy()
    for row in cleaned_data:
        if 'Min salary' not in row or 'Max salary' not in row:
            continue
        row['Average salary'] = (row['Min salary'] + row['Max salary'])/2
    return cleaned_data            

In [149]:
data = calculate_average_salary_as_new_column(data)

## Make job titles more consistent

In [151]:
def replace_job_title_with_consistent_naming(data, col, value_to_replace, new_value):
    cleaned_data = data.copy()
    for dictionary in cleaned_data:
        for key, value in list(dictionary.items()):
            if key == col:
                value = str(value).replace(value_to_replace, new_value)
                dictionary[key] = value     

In [152]:
replace_job_title_with_consistent_naming(data, 'Job Title', 'Entry Level / Junior Data Analyst', 'Junior Data Analyst')

In [153]:
replace_job_title_with_consistent_naming(data, 'Job Title', 'Data Analyst â Junior', 'Junior Data Analyst')

In [154]:
replace_job_title_with_consistent_naming(data, 'Job Title', 'Entry-Level Data Analyst', 'Junior Data Analyst')

In [155]:
replace_job_title_with_consistent_naming(data, 'Job Title', 'Entry Level Data Analyst', 'Junior Data Analyst')

In [156]:
replace_job_title_with_consistent_naming(data, 'Job Title', 'Data Analyst Entry Level', 'Junior Data Analyst')

In [157]:
replace_job_title_with_consistent_naming(data, 'Job Title', 'Jr. Data Analyst', 'Junior Data Analyst')

In [158]:
replace_job_title_with_consistent_naming(data, 'Job Title', 'Jr Data Analyst', 'Junior Data Analyst')

In [159]:
replace_job_title_with_consistent_naming(data, 'Job Title', 'Data Analyst Junior', 'Junior Data Analyst')

In [160]:
replace_job_title_with_consistent_naming(data, 'Job Title', 'Sr Data Analyst', 'Senior Data Analyst')

In [161]:
replace_job_title_with_consistent_naming(data, 'Job Title', 'Sr. Data Analyst', 'Senior Data Analyst')

In [162]:
replace_job_title_with_consistent_naming(data, 'Job Title', 'Data Analyst Senior', 'Senior Data Analyst')

In [163]:
replace_job_title_with_consistent_naming(data, 'Job Title', 'DATA ANALYST', 'Data Analyst')

In [164]:
replace_job_title_with_consistent_naming(data, 'Job Title', 'Sr.', 'Senior')

In [165]:
replace_job_title_with_consistent_naming(data, 'Job Title', 'Sr', 'Senior')

In [166]:
replace_job_title_with_consistent_naming(data, 'Job Title', 'SR.', 'Senior')

In [167]:
replace_job_title_with_consistent_naming(data, 'Job Title', 'SR', 'Senior')

In [168]:
replace_job_title_with_consistent_naming(data, 'Job Title', 'Jr', 'Junior')

In [169]:
replace_job_title_with_consistent_naming(data, 'Job Title', 'Jr.', 'Junior')

In [170]:
replace_job_title_with_consistent_naming(data, 'Job Title', 'JR.', 'Junior')

In [171]:
replace_job_title_with_consistent_naming(data, 'Job Title', 'JR', 'Junior')

## Extract seniority level

In [172]:
jobs = find_unique_values_for_column(data, 'Job Title')

In [175]:
import re
def extract_keyword_as_column(data, col, pattern, new_header):
    cleaned_data = data.copy()
    for row in cleaned_data:
        if re.search(pattern, row[col], re.IGNORECASE):
            row[new_header] = True
        else:
            row[new_header] = False
    return cleaned_data        

In [176]:
def extract_seniority_as_column(data, col):
    seniority_keywords = {'Junior level':'[\W*]Junior|Jr|Entry[\s-]level|Intern[\W*]', 'Senior level':'[\W*]Senior|Sr[\W*]','Lead level':'[\W*]Lead|Principal[\W*]'}
    for new_header, pattern in seniority_keywords.items():
        data = extract_keyword_as_column(data, col, pattern, new_header)
    return data   

In [177]:
data = extract_seniority_as_column(data, 'Job Title')

In [178]:
def extract_mid_seniority(data):
    for row in data:
        if row['Junior level'] == False and row['Senior level'] == False and row['Lead level'] == False:
            row['Mid level'] = True
        else:
            row['Mid level'] = False
    return data        

In [179]:
data = extract_mid_seniority(data)

## Extract skills and education keywords

In [181]:
def extract_skills_as_columns(data, col):
    skill_keywords = {'Excel required':'[\W+]Excel[\W+]', 'SQL required':'[\W+]SQL[\W+]','Python required':'[\W+]Python[\W+]','R required':'[\W+]R[\W+]','Tableau required':'[\W+]Tableau[\W+]',
                      'PowerBI required':'[\W+]Power[\s*]BI[\W+]'}
    for new_header, pattern in skill_keywords.items():
        data = extract_keyword_as_column(data, col, pattern, new_header)
    return data    

In [182]:
data = extract_skills_as_columns(data, 'Job Description')

In [183]:
print_no_job_description(data)

Job Title:Data Analyst, Center on Immigration and Justice (CIJ)
Salary Estimate:$37K-$66K (Glassdoor est.)
Rating:3.2
Company Name:Vera Institute of Justice
Location:New York, NY
Headquarters:New York, NY
Size:201 to 500 employees
Founded:1961
Type of ownership:Nonprofit Organization
Industry:Social Assistance
Sector:Non-Profit
Revenue:$100 to $500 million (USD)
Competitors:NA
Easy Apply:True
Index:0
Max size:500
Min salary:37
Max salary:66
Average salary:51.5
Junior level:False
Senior level:False
Lead level:False
Mid level:True
Excel required:False
SQL required:True
Python required:True
R required:True
Tableau required:False
PowerBI required:False


Job Title:Quality Data Analyst
Salary Estimate:$37K-$66K (Glassdoor est.)
Rating:3.8
Company Name:Visiting Nurse Service of New York
Location:New York, NY
Headquarters:New York, NY
Size:10000+ employees
Founded:1893
Type of ownership:Nonprofit Organization
Industry:Health Care Services & Hospitals
Sector:Health Care
Revenue:$2 to $5 billio

In [ ]:
def extract_education_as_columns(data, col):
    skill_keywords = {'Bachelor':'[\W+]Bachelor|BSc[\W+]', 'Doctorate':'[\W+]Doctor|PhD[\W+]', 'Master':'[\W+]Master|MSc[\W+]'}
    for new_header, pattern in skill_keywords.items():
        data = extract_keyword_as_column(data, col, pattern, new_header)
    return data    

In [ ]:
data = extract_education_as_columns(data, 'Job Description')

## Count and handle missing values

In [184]:
def count_missing_keys(data, col):
    count_missing_keys = 0
    for row in data:
        if col not in row:
            count_missing_keys += 1
    return count_missing_keys        

In [185]:
def count_missing_keys_info(data):
    all_headers = get_all_headers(data)
    missing_info = {}
    for header in all_headers:
        missing_info[header] = count_missing_keys(data, header)
    return missing_info    

In [186]:
count_missing_keys_info(data)

{'Excel required': 0,
 'R required': 0,
 'Industry': 0,
 'Revenue': 0,
 'Size': 0,
 'Sector': 0,
 'Max salary': 1,
 'Competitors': 0,
 'PowerBI required': 0,
 'Headquarters': 0,
 'Max size': 0,
 'Mid level': 0,
 'Founded': 0,
 'Python required': 0,
 'Rating': 0,
 'Lead level': 0,
 'Average salary': 1,
 'Company Name': 0,
 'Senior level': 0,
 'Index': 0,
 'Salary Estimate': 0,
 'Job Description': 0,
 'Easy Apply': 0,
 'Min salary': 1,
 'Tableau required': 0,
 'Location': 0,
 'Junior level': 0,
 'Type of ownership': 0,
 'SQL required': 0,
 'Job Title': 0}

In [187]:
def replace_missing_keys_with_NA(data, col):
    for row in data:
        if col not in row:
            row[col] = "NA"
    return data        

In [188]:
data = replace_missing_keys_with_NA(data, 'Min salary')

In [189]:
data = replace_missing_keys_with_NA(data, 'Max salary')

In [190]:
data = replace_missing_keys_with_NA(data, 'Average salary')

## Check the relevance of each column

In [191]:
def count_NA_values_for_specified_column(data, col):
    count_invalid_data = 0 
    for row in data:
        if col not in row:
            continue
        if row[col] == 'NA':
            count_invalid_data += 1
    return count_invalid_data       

In [192]:
def count_NA_values(data):
    all_headers = get_all_headers(data)
    NA_values_per_column = {}
    for header in all_headers:
        NA_values_per_column[header] = count_NA_values_for_specified_column(data, header)
    return NA_values_per_column

In [193]:
count_NA_values(data)

{'Excel required': 0,
 'R required': 0,
 'Industry': 353,
 'Revenue': 778,
 'Size': 205,
 'Sector': 353,
 'Max salary': 1,
 'Competitors': 1732,
 'PowerBI required': 0,
 'Headquarters': 172,
 'Max size': 205,
 'Mid level': 0,
 'Founded': 660,
 'Python required': 0,
 'Rating': 272,
 'Lead level': 0,
 'Average salary': 1,
 'Company Name': 1,
 'Senior level': 0,
 'Index': 0,
 'Salary Estimate': 1,
 'Job Description': 0,
 'Easy Apply': 2173,
 'Min salary': 1,
 'Tableau required': 0,
 'Location': 0,
 'Junior level': 0,
 'Type of ownership': 179,
 'SQL required': 0,
 'Job Title': 0}

## Write data as a csv file

In [ ]:
def create_file(data):
    result = []
    for row in data:
       
            new_info = {}
            new_info['Job title'] = row['Job Title']
            new_info['Company name'] = row['Company Name']
            new_info['Rating'] = row['Rating']
            new_info['Location'] = row['Location']
            new_info['Max size'] = row['Max size']
            new_info['Founded'] = row['Founded']
            new_info['Type of ownership'] = row['Type of ownership']
            new_info['Industry'] = row['Industry']
            new_info['Minimum estimated salary'] = row['Min salary']
            new_info['Maximum estimated salary'] = row['Max salary']
            new_info['Average estimated salary'] = row['Average salary']
            new_info['Excel required'] = row['Excel required']
            new_info['SQL required'] = row['SQL required']
            new_info['R required'] = row['R required']
            new_info['Python required'] = row['Python required']
            new_info['Tableau required'] = row['Tableau required']
            new_info['PowerBI required'] = row['PowerBI required']
            new_info['Junior level'] = row['Junior level']
            new_info['Mid level'] = row['Mid level']
            new_info['Senior level'] = row['Senior level']
            new_info['Lead level'] = row['Lead level']
            new_info['Bachelor'] = row['Bachelor']
            new_info['Master'] = row['Master']
            new_info['Doctorate'] = row['Doctorate']
            result.append(new_info)    
    return result    

In [ ]:
new_data = create_file(data)

In [ ]:
print(len(data))
print(len(new_data))

In [ ]:
# write output to csv file
def write_as_csv(new_data):
    
    file = open('data_analyst_modified.csv', 'w', newline='', encoding="utf8")
    dict_writer = csv.DictWriter(file, fieldnames=['Job title','Company name', 'Max size','Rating', 'Location', 'Founded','Type of ownership','Industry', 'Minimum estimated salary',
                                                   'Maximum estimated salary','Average estimated salary', 'Excel required', 'SQL required', 'R required', 'Python required','Tableau required', 
                                                   'PowerBI required', 'Junior level', 'Mid level','Senior level','Lead level','Bachelor', 'Master','Doctorate'])
    dict_writer.writeheader()
    for row in new_data:
        dict_writer.writerow(row)
    file.close()

In [ ]:
write_as_csv(new_data)